In [2]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog
import math
import time
import highspy as hi

#on va écrire le problème sous la forme "min c^T x tel que Ax >= b et x >= 0" où c est le vecteur coût et x = x_t + y_t
#avec x_t la puissance par la pompe en temps normal et y_t la puissance par la pompe en temps reverse

#différents paramètres
nb_intervalles = 672 #7*24*4
temperatures_ext = np.load("Temperatures-Montreal.npy")
eta = 0.99
capacite_calorifique = 0.4 * 360 #en kWh
T_min = 19
T_max = 21
T_int = cp.Variable(nb_intervalles)
cost_electricity = np.array([0.18 if (i * 4) % 24 < 7 or (i * 4) % 24 >= 22 else 0.26 for i in range(nb_intervalles)]) #coût qui est variable
P_chauff = cp.Variable(nb_intervalles) #puissance par la pompe en temps normal
P_refroid = cp.Variable(nb_intervalles)#puissance par la pompe en mode inverse

contraintes = [] #matrice A
#début de la résolution, il ne faut pas oubleir que T_0 = T_672 qui doivent etre entre T_min et T_max
for i in range(nb_intervalles):
    contraintes.append(T_int[i] >= T_min)
    contraintes.append(T_int[i] <= T_max)

for i in range(nb_intervalles - 1):
    COP_chauff = 3 + 10 * np.abs(np.tanh(temperatures_ext[i] / 100)) * np.tanh(temperatures_ext[i] / 100)
    COP_refroid = 3.2  # valeur constante pour COP_refroid

    contraintes.append(T_int[i + 1] - T_int[i] - (1 - eta) * (temperatures_ext[i] - T_int[i]))

#On finit les contraintes
contraintes.append(P_chauff >= 0) #puissances positives
contraintes.append(P_refroid >= 0)

cost = cp.sum(cost_electricity * (P_chauff + P_refroid)) #coût total
start_time = time.time()
res = cp.Problem(cp.Minimize(cost), contraintes)

#res = _highs.(cost, contraintes_ub=contraintes, b_ub=b, options={"disp": False}) #ici je sais pas trop comment faire
temps_calcul = time.time() - start_time
cout_min = res.value; T_int = T_int.value; P_chauff = P_chauff.value; P_refroid = P_refroid.value;
# Résoudre le problème pour les deux périodes
resultats_periode_1 = resoudre_probleme_optimisation(periode_1)
resultats_periode_2 = resoudre_probleme_optimisation(periode_2)

# Afficher les résultats
print(f"Période 1 - Coût minimal: {resultats_periode_1['cout_minimal']:.2f}, Temps de calcul: {resultats_periode_1['temps_calcul']:.2f} s")
print(f"Période 2 - Coût minimal: {resultats_periode_2['cout_minimal']:.2f}, Temps de calcul: {resultats_periode_2['temps_calcul']:.2f} s")

# Graphique de l'évolution des températures
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(resultats_periode_1["T_int"])
plt.title("Période 1 - Évolution des températures")
plt.xlabel("Intervalle de temps")
plt.ylabel("Température (°C)")

plt.subplot(1, 2, 2)
plt.plot(resultats_periode_2["T_int"])
plt.title("Période 2 - Évolution des températures")
plt.xlabel("Intervalle de temps")
plt.ylabel("Température (°C)")

plt.show()

# Graphique représentant l'utilisation de la pompe à chaleur
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(resultats_periode_1["P_chauff"], label="Fonctionnement normal")
plt.plot(resultats_periode_1["P_refroid"], label="Fonctionnement reverse")
plt.title("Période 1 - Utilisation de la pompe à chaleur")
plt.xlabel("Intervalle de temps")
plt.ylabel("Puissance (kW)")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(resultats_periode_2["P_chauff"], label="Fonctionnement normal")
plt.plot(resultats_periode_2["P_refroid"], label="Fonctionnement reverse")
plt.title("Période 2 - Utilisation de la pompe à chaleur")
plt.xlabel("Intervalle de temps")
plt.ylabel("Puissance (kW)")
plt.legend()

plt.show()

# Bref commentaire
print("Les graphiques montrent que l'optimisation permet de minimiser les coûts tout en gérant l'inconfort. Les températures restent proches des limites admissibles et l'utilisation de la pompe à chaleur varie en fonction des besoins. Les deux périodes présentent des comportements différents, soulignant l'importance de choisir des périodes intéressantes pour l'analyse.")




#DEUXIÈME TÂCHE À PARTIR D'ICI 

import cvxpy as cp #conseillé par l'énoncé
import numpy as np #vecteurs et matrices
import matplotlib.pyplot as plt #pour les graphes
from scipy.optimize import _highs #solver conseillé
import math #pour les tanh mais apparemment numpy les import déjà

#on va écrire le problème sous la forme "min c^T x tel que Ax >= b et x >= 0" où c est le vecteur coût et x = x_t + y_t
#on va minimiser l'inconfort total cette fois sauf que cette fois on va enlever la contrainte que
#T_0 = T_672 appartiennent à [19,21]
#avec x_t la puissance par la pompe en temps normal et y_t la puissance par la pompe en temps reverse

#différents paramètres
nb_intervalles = 7*24*4
temperatures_ext = np.load("Temperatures-Montreal.npy")
eta = 0.99
capacite_calorifique = 0.4 * 360 #en kWh
T_min = 19
T_max = 21
T_int = cp.Variable(nb_intervalles)
cost_electricity = np.array([0.18 if (i * 4) % 24 < 7 or (i * 4) % 24 >= 22 else 0.26 for i in range(nb_intervalles)]) #coût qui est variable
P_chauff = cp.Variable(nb_intervalles) #puissance par la pompe en temps normal
P_refroid = cp.Variable(nb_intervalles)#puissance par la pompe en mode inverse
inconfort_inf = cp.Variable(nb_intervalles, nonneg=True)  # inconfort pour les températures inférieures à T_min
inconfort_sup = cp.Variable(nb_intervalles, nonneg=True)  # inconfort pour les températures supérieures à T_max
penalite_inf = 3  # pénalité pour chaque degré en dessous de T_min
penalite_sup = 1  # pénalité pour chaque degré au-dessus de T_max

budget_maximal = 10 #À TROUVER!!!!

contraintes = [] #matrice A
#début de la résolution
#Ici les températures ne doivent plus appartenir à un certain intevevalle mais
#si jamais ça dépasse, des coefficients de pénalité apparaissent
for i in range(nb_intervalles):
    contraintes.append(T_int[i] - T_min >= -inconfort_inf[i]) #Cette contrainte s'assure que si la température intérieure T_int[i] est inférieure à T_min, alors la différence T_int[i] - T_min sera négative, et l'inconfort correspondant à cette différence sera stocké dans la variable inconfort_inf[i]. Le moins devant c'est car ça sera une valeur négative or on prend que les positives
    contraintes.append(T_int[i] - T_max <= inconfort_sup[i])


for i in range(nb_intervalles - 1):
    COP_chauff = 3 + 10 * np.abs(np.tanh(temperatures_ext[i] / 100)) * np.tanh(temperatures_ext[i] / 100)
    COP_refroid = 3.2  # valeur constante pour COP_refroid

    contraintes.append(T_int[i + 1] - T_int[i] - (1 - eta) * (temperatures_ext[i] - T_int[i]))

#On finit les contraintes
contraintes.append(P_chauff >= 0) #puissances positives
contraintes.append(P_refroid >= 0)
total_inconfort = cp.sum(penalite_inf * inconfort_inf + penalite_sup * inconfort_sup)
cost = cp.sum(cost_electricity * (P_chauff + P_refroid)) #coût total
contraintes.append(cost <= budget_maximal)  # contrainte de budget qu'il faudrait avoir trouvé à la question 1


start_time = time.time()
#res = cp.Problem(cp.Minimize(cost), contraintes)

#res = _highs.(cost, contraintes_ub=contraintes, b_ub=b, options={"disp": False}) #ici je sais pas trop comment faire
temps_calcul = time.time() - start_time
cout_min = res.value; T_int = T_int.value; P_chauff = P_chauff.value; P_refroid = P_refroid.value;
# Résoudre le problème pour les deux périodes
resultats_periode_1 = resoudre_probleme_optimisation(periode_1)
resultats_periode_2 = resoudre_probleme_optimisation(periode_2)

# Afficher les résultats
print(f"Période 1 - Coût minimal: {resultats_periode_1['cout_minimal']:.2f}, Temps de calcul: {resultats_periode_1['temps_calcul']:.2f} s")
print(f"Période 2 - Coût minimal: {resultats_periode_2['cout_minimal']:.2f}, Temps de calcul: {resultats_periode_2['temps_calcul']:.2f} s")

# Graphique de l'évolution des températures
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(resultats_periode_1["T_int"])
plt.title("Période 1 - Évolution des températures")
plt.xlabel("Intervalle de temps")
plt.ylabel("Température (°C)")

plt.subplot(1, 2, 2)
plt.plot(resultats_periode_2["T_int"])
plt.title("Période 2 - Évolution des températures")
plt.xlabel("Intervalle de temps")
plt.ylabel("Température (°C)")

plt.show()

# Graphique représentant l'utilisation de la pompe à chaleur
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(resultats_periode_1["P_chauff"], label="Fonctionnement normal")
plt.plot(resultats_periode_1["P_refroid"], label="Fonctionnement reverse")
plt.title("Période 1 - Utilisation de la pompe à chaleur")
plt.xlabel("Intervalle de temps")
plt.ylabel("Puissance (kW)")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(resultats_periode_2["P_chauff"], label="Fonctionnement normal")
plt.plot(resultats_periode_2["P_refroid"], label="Fonctionnement reverse")
plt.title("Période 2 - Utilisation de la pompe à chaleur")
plt.xlabel("Intervalle de temps")
plt.ylabel("Puissance (kW)")
plt.legend()

plt.show()

# Bref commentaire
print("Les graphiques montrent que l'optimisation permet de minimiser les coûts tout en gérant l'inconfort. Les températures restent proches des limites admissibles et l'utilisation de la pompe à chaleur varie en fonction des besoins. Les deux périodes présentent des comportements différents, soulignant l'importance de choisir des périodes intéressantes pour l'analyse.")


/home/iss25/.local/lib/python3.10/site-packages/cvxpy/expressions/expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)


NameError: name 'res' is not defined